In [1]:
"""
Global Trade Model Based on Labour Theory of Value
====================================================
Generates synthetic but realistic import/export data for all countries
across commodity groups over multiple years.

Core LTV Assumptions:
- Value of a commodity is proportional to socially necessary labour time (SNLT)
- Countries have different labour productivities per sector
- Trade arises from differences in labour costs & resource endowments
- Countries export goods where their labour is relatively more productive
  (comparative advantage expressed through labour values)
"""

import numpy as np
import pandas as pd
import json
from dataclasses import dataclass, field
from typing import Dict, List, Tuple

# ============================================================
# 1. COUNTRY DATABASE
# ============================================================

# All UN-recognized countries + major trading territories
COUNTRIES = {
    # Format: ISO3 -> (Name, Population_millions_2023, GDP_nominal_billions_2023,
    #                   Labour_force_participation_rate, Industrialization_index 0-1)
    "USA": ("United States", 335, 25500, 0.61, 0.88),
    "CHN": ("China", 1425, 17900, 0.68, 0.82),
    "JPN": ("Japan", 125, 4230, 0.62, 0.91),
    "DEU": ("Germany", 84, 4070, 0.62, 0.92),
    "GBR": ("United Kingdom", 67, 3070, 0.62, 0.85),
    "IND": ("India", 1425, 3390, 0.46, 0.52),
    "FRA": ("France", 68, 2780, 0.56, 0.84),
    "ITA": ("Italy", 59, 2010, 0.50, 0.83),
    "BRA": ("Brazil", 215, 1920, 0.57, 0.58),
    "CAN": ("Canada", 39, 2140, 0.64, 0.80),
    "RUS": ("Russia", 144, 1860, 0.59, 0.68),
    "KOR": ("South Korea", 52, 1670, 0.63, 0.90),
    "AUS": ("Australia", 26, 1680, 0.65, 0.78),
    "MEX": ("Mexico", 128, 1320, 0.57, 0.62),
    "ESP": ("Spain", 48, 1400, 0.58, 0.78),
    "IDN": ("Indonesia", 277, 1320, 0.67, 0.50),
    "NLD": ("Netherlands", 18, 1010, 0.65, 0.87),
    "SAU": ("Saudi Arabia", 36, 1060, 0.58, 0.55),
    "TUR": ("Turkey", 85, 906, 0.52, 0.63),
    "CHE": ("Switzerland", 9, 870, 0.68, 0.93),
    "POL": ("Poland", 38, 690, 0.57, 0.72),
    "SWE": ("Sweden", 10, 590, 0.64, 0.88),
    "BEL": ("Belgium", 12, 580, 0.54, 0.85),
    "NOR": ("Norway", 5, 580, 0.65, 0.82),
    "AUT": ("Austria", 9, 470, 0.62, 0.85),
    "IRL": ("Ireland", 5, 530, 0.62, 0.86),
    "ISR": ("Israel", 9, 520, 0.63, 0.87),
    "ARG": ("Argentina", 46, 630, 0.57, 0.55),
    "SGP": ("Singapore", 6, 424, 0.68, 0.90),
    "THA": ("Thailand", 72, 535, 0.67, 0.64),
    "ARE": ("UAE", 10, 509, 0.77, 0.65),
    "MYS": ("Malaysia", 34, 407, 0.64, 0.68),
    "PHL": ("Philippines", 115, 404, 0.59, 0.48),
    "EGY": ("Egypt", 111, 395, 0.42, 0.45),
    "VNM": ("Vietnam", 99, 409, 0.76, 0.60),
    "BGD": ("Bangladesh", 171, 420, 0.57, 0.45),
    "NGA": ("Nigeria", 223, 477, 0.53, 0.32),
    "CHL": ("Chile", 19, 335, 0.59, 0.58),
    "COL": ("Colombia", 52, 335, 0.62, 0.48),
    "CZE": ("Czech Republic", 11, 290, 0.60, 0.80),
    "PER": ("Peru", 34, 245, 0.72, 0.42),
    "ROU": ("Romania", 19, 287, 0.53, 0.65),
    "NZL": ("New Zealand", 5, 247, 0.67, 0.72),
    "PRT": ("Portugal", 10, 255, 0.58, 0.72),
    "KAZ": ("Kazakhstan", 19, 220, 0.69, 0.48),
    "QAT": ("Qatar", 3, 221, 0.82, 0.55),
    "HUN": ("Hungary", 10, 184, 0.58, 0.73),
    "GRC": ("Greece", 10, 220, 0.52, 0.62),
    "DNK": ("Denmark", 6, 400, 0.63, 0.86),
    "FIN": ("Finland", 6, 282, 0.62, 0.86),
    "ZAF": ("South Africa", 60, 399, 0.42, 0.52),
    "PAK": ("Pakistan", 230, 340, 0.51, 0.38),
    "DZA": ("Algeria", 45, 187, 0.40, 0.40),
    "IRQ": ("Iraq", 43, 267, 0.40, 0.30),
    "MAR": ("Morocco", 37, 133, 0.44, 0.45),
    "KWT": ("Kuwait", 4, 165, 0.72, 0.48),
    "ECU": ("Ecuador", 18, 115, 0.65, 0.38),
    "SVK": ("Slovakia", 5, 115, 0.58, 0.73),
    "OMN": ("Oman", 5, 105, 0.68, 0.45),
    "GTM": ("Guatemala", 18, 95, 0.58, 0.32),
    "LKA": ("Sri Lanka", 22, 75, 0.51, 0.42),
    "KEN": ("Kenya", 55, 113, 0.60, 0.30),
    "ETH": ("Ethiopia", 120, 156, 0.78, 0.18),
    "TZA": ("Tanzania", 65, 79, 0.82, 0.18),
    "UKR": ("Ukraine", 37, 160, 0.56, 0.55),
    "MMR": ("Myanmar", 54, 59, 0.62, 0.30),
    "DOM": ("Dominican Republic", 11, 114, 0.60, 0.42),
    "AGO": ("Angola", 35, 107, 0.65, 0.25),
    "HRV": ("Croatia", 4, 70, 0.52, 0.65),
    "BGR": ("Bulgaria", 7, 100, 0.54, 0.62),
    "UZB": ("Uzbekistan", 35, 80, 0.60, 0.38),
    "PAN": ("Panama", 4, 77, 0.65, 0.42),
    "TUN": ("Tunisia", 12, 47, 0.47, 0.48),
    "LTU": ("Lithuania", 3, 67, 0.60, 0.70),
    "GHA": ("Ghana", 33, 73, 0.67, 0.28),
    "SVN": ("Slovenia", 2, 63, 0.58, 0.78),
    "CRI": ("Costa Rica", 5, 69, 0.58, 0.48),
    "URY": ("Uruguay", 4, 72, 0.60, 0.52),
    "LBN": ("Lebanon", 5, 22, 0.46, 0.38),
    "SRB": ("Serbia", 7, 63, 0.54, 0.58),
    "LVA": ("Latvia", 2, 41, 0.58, 0.65),
    "EST": ("Estonia", 1, 38, 0.62, 0.72),
    "CIV": ("Côte d'Ivoire", 28, 70, 0.56, 0.25),
    "CMR": ("Cameroon", 28, 45, 0.70, 0.22),
    "BOL": ("Bolivia", 12, 44, 0.68, 0.32),
    "PRY": ("Paraguay", 7, 42, 0.65, 0.32),
    "JOR": ("Jordan", 11, 47, 0.36, 0.42),
    "BHR": ("Bahrain", 2, 44, 0.72, 0.50),
    "TTO": ("Trinidad and Tobago", 1, 28, 0.58, 0.48),
    "SEN": ("Senegal", 17, 28, 0.48, 0.22),
    "ZWE": ("Zimbabwe", 16, 30, 0.80, 0.22),
    "COD": ("DR Congo", 100, 64, 0.60, 0.12),
    "MOZ": ("Mozambique", 33, 17, 0.75, 0.15),
    "ZMB": ("Zambia", 20, 29, 0.72, 0.22),
    "MLI": ("Mali", 22, 19, 0.65, 0.12),
    "BFA": ("Burkina Faso", 22, 19, 0.62, 0.10),
    "MDG": ("Madagascar", 29, 15, 0.83, 0.12),
    "NER": ("Niger", 26, 15, 0.60, 0.08),
    "TCD": ("Chad", 17, 12, 0.62, 0.10),
    "MWI": ("Malawi", 20, 13, 0.76, 0.10),
    "RWA": ("Rwanda", 14, 13, 0.83, 0.15),
    "KHM": ("Cambodia", 17, 30, 0.82, 0.35),
    "LAO": ("Laos", 7, 15, 0.77, 0.25),
    "NPL": ("Nepal", 30, 40, 0.80, 0.20),
    "HND": ("Honduras", 10, 31, 0.58, 0.30),
    "SLV": ("El Salvador", 6, 33, 0.56, 0.35),
    "NIC": ("Nicaragua", 7, 17, 0.60, 0.28),
    "BIH": ("Bosnia and Herzegovina", 3, 24, 0.42, 0.48),
    "ALB": ("Albania", 3, 18, 0.54, 0.38),
    "MKD": ("North Macedonia", 2, 14, 0.48, 0.42),
    "MNE": ("Montenegro", 1, 6, 0.50, 0.38),
    "GEO": ("Georgia", 4, 25, 0.56, 0.38),
    "ARM": ("Armenia", 3, 19, 0.55, 0.38),
    "AZE": ("Azerbaijan", 10, 54, 0.63, 0.42),
    "TKM": ("Turkmenistan", 6, 45, 0.62, 0.32),
    "KGZ": ("Kyrgyzstan", 7, 12, 0.58, 0.25),
    "TJK": ("Tajikistan", 10, 11, 0.55, 0.20),
    "MNG": ("Mongolia", 3, 17, 0.58, 0.30),
    "CUB": ("Cuba", 11, 107, 0.50, 0.35),
    "HTI": ("Haiti", 12, 20, 0.60, 0.12),
    "JAM": ("Jamaica", 3, 17, 0.63, 0.32),
    "CYP": ("Cyprus", 1, 28, 0.62, 0.58),
    "LUX": ("Luxembourg", 1, 87, 0.58, 0.82),
    "MLT": ("Malta", 1, 18, 0.60, 0.65),
    "ISL": ("Iceland", 0.4, 28, 0.78, 0.72),
    "BRN": ("Brunei", 0.4, 14, 0.58, 0.48),
    "UGA": ("Uganda", 47, 46, 0.70, 0.15),
    "SDN": ("Sudan", 47, 34, 0.45, 0.15),
    "LBY": ("Libya", 7, 42, 0.48, 0.30),
    "YEM": ("Yemen", 33, 21, 0.35, 0.10),
    "SYR": ("Syria", 22, 11, 0.40, 0.18),
    "AFG": ("Afghanistan", 40, 14, 0.45, 0.10),
    "SOM": ("Somalia", 17, 8, 0.50, 0.06),
    "BEN": ("Benin", 13, 18, 0.68, 0.15),
    "TGO": ("Togo", 9, 8, 0.72, 0.15),
    "SLE": ("Sierra Leone", 8, 4, 0.55, 0.08),
    "LBR": ("Liberia", 5, 4, 0.55, 0.08),
    "MRT": ("Mauritania", 5, 10, 0.48, 0.18),
    "GAB": ("Gabon", 2, 20, 0.55, 0.30),
    "BWA": ("Botswana", 3, 19, 0.62, 0.35),
    "NAM": ("Namibia", 3, 12, 0.55, 0.30),
    "MUS": ("Mauritius", 1, 14, 0.55, 0.48),
    "SWZ": ("Eswatini", 1, 5, 0.52, 0.25),
    "LSO": ("Lesotho", 2, 3, 0.55, 0.18),
    "FJI": ("Fiji", 1, 5, 0.55, 0.28),
    "GUY": ("Guyana", 1, 15, 0.55, 0.30),
    "SUR": ("Suriname", 1, 4, 0.52, 0.28),
    "BLZ": ("Belize", 0.4, 2, 0.60, 0.25),
    "MDA": ("Moldova", 3, 14, 0.42, 0.35),
    "BLR": ("Belarus", 9, 70, 0.62, 0.58),
    "PNG": ("Papua New Guinea", 10, 31, 0.68, 0.18),
    "TWN": ("Taiwan", 24, 790, 0.59, 0.90),
    "HKG": ("Hong Kong", 7, 360, 0.58, 0.80),
}


# ============================================================
# 2. COMMODITY GROUPS (with LTV characteristics)
# ============================================================

@dataclass
class CommodityGroup:
    """
    Each commodity group has:
    - name: Human-readable name
    - base_snlt: Base socially necessary labour time index (hours per \$1000 of value)
    - capital_intensity: Ratio of constant capital to variable capital (c/v)
    - resource_dependence: How much natural resource access matters (0-1)
    - skill_intensity: How much skilled labour matters (0-1)
    - bulk_weight: Affects transport costs (0-1 scale)
    - global_demand_billions: Approximate total global trade in this category
    - demand_elasticity: Price elasticity of demand
    - growth_trend: Annual growth rate trend
    """
    name: str
    base_snlt: float
    capital_intensity: float
    resource_dependence: float
    skill_intensity: float
    bulk_weight: float
    global_demand_billions: float
    demand_elasticity: float
    growth_trend: float


COMMODITY_GROUPS = {
    "AGR": CommodityGroup(
        "Agricultural Products & Foodstuffs", 12.0, 0.4, 0.8, 0.2, 0.7,
        1800, 0.3, 0.03
    ),
    "OIL": CommodityGroup(
        "Crude Oil & Petroleum Products", 2.0, 0.9, 0.95, 0.3, 0.9,
        2400, 0.4, -0.01
    ),
    "GAS": CommodityGroup(
        "Natural Gas & LNG", 2.5, 0.9, 0.95, 0.3, 0.8,
        600, 0.35, 0.04
    ),
    "MIN": CommodityGroup(
        "Ores, Metals & Minerals", 5.0, 0.8, 0.9, 0.3, 0.9,
        1200, 0.5, 0.03
    ),
    "CHM": CommodityGroup(
        "Chemicals & Petrochemicals", 4.0, 0.8, 0.5, 0.6, 0.6,
        1400, 0.5, 0.04
    ),
    "PHA": CommodityGroup(
        "Pharmaceuticals", 1.5, 0.7, 0.2, 0.9, 0.1,
        800, 0.2, 0.06
    ),
    "MED": CommodityGroup(
        "Medical Instruments & Equipment", 2.0, 0.7, 0.1, 0.9, 0.2,
        450, 0.3, 0.07
    ),
    "TEX": CommodityGroup(
        "Textiles, Apparel & Footwear", 15.0, 0.3, 0.3, 0.2, 0.3,
        900, 0.7, 0.03
    ),
    "AUT": CommodityGroup(
        "Automobiles & Auto Parts", 3.0, 0.85, 0.2, 0.6, 0.7,
        1600, 0.8, 0.03
    ),
    "ELE": CommodityGroup(
        "Electronics & Semiconductors", 1.0, 0.9, 0.1, 0.9, 0.1,
        2800, 0.6, 0.06
    ),
    "MAC": CommodityGroup(
        "Industrial Machinery & Equipment", 3.5, 0.8, 0.15, 0.7, 0.5,
        1500, 0.6, 0.04
    ),
    "AIR": CommodityGroup(
        "Aerospace & Aircraft", 1.5, 0.9, 0.1, 0.95, 0.3,
        350, 0.5, 0.04
    ),
    "STL": CommodityGroup(
        "Iron, Steel & Fabricated Metals", 5.0, 0.85, 0.6, 0.4, 0.9,
        800, 0.6, 0.02
    ),
    "PLR": CommodityGroup(
        "Plastics & Rubber Products", 5.0, 0.7, 0.4, 0.4, 0.5,
        700, 0.5, 0.03
    ),
    "WDP": CommodityGroup(
        "Wood, Paper & Forestry Products", 8.0, 0.5, 0.7, 0.2, 0.8,
        450, 0.4, 0.01
    ),
    "FRT": CommodityGroup(
        "Fertilizers & Agricultural Chemicals", 4.0, 0.75, 0.7, 0.4, 0.8,
        250, 0.3, 0.03
    ),
    "GEM": CommodityGroup(
        "Precious Stones & Jewelry", 0.8, 0.3, 0.85, 0.5, 0.05,
        350, 1.0, 0.02
    ),
    "FUR": CommodityGroup(
        "Furniture & Household Goods", 10.0, 0.4, 0.3, 0.3, 0.6,
        500, 0.8, 0.04
    ),
    "OPT": CommodityGroup(
        "Optical & Precision Instruments", 1.5, 0.8, 0.1, 0.9, 0.1,
        300, 0.4, 0.05
    ),
    "SHP": CommodityGroup(
        "Ships, Boats & Marine Equipment", 4.0, 0.85, 0.2, 0.6, 0.95,
        200, 0.7, 0.02
    ),
    "ARM": CommodityGroup(
        "Arms & Defense Equipment", 2.0, 0.9, 0.15, 0.85, 0.4,
        120, 0.2, 0.03
    ),
    "TOY": CommodityGroup(
        "Toys, Games & Sporting Goods", 12.0, 0.4, 0.1, 0.2, 0.3,
        180, 0.9, 0.03
    ),
    "CER": CommodityGroup(
        "Cement, Glass & Ceramics", 7.0, 0.7, 0.6, 0.3, 0.95,
        250, 0.4, 0.03
    ),
}


# ============================================================
# 3. COUNTRY ENDOWMENT PROFILES
# ============================================================

def build_country_endowments(countries: dict, seed=42) -> pd.DataFrame:
    """
    For each country, generate endowment scores (0-1) for each factor
    that drives trade patterns:
    - natural_resources: oil, minerals, arable land
    - labour_abundance: cheap labour availability
    - labour_skill: education, technical training
    - capital_stock: accumulated industrial capital
    - infrastructure: ports, roads, logistics
    - tech_level: R&D capacity, innovation

    These are semi-deterministic: major countries get hand-tuned profiles,
    smaller ones are generated from GDP/pop heuristics + noise.
    """
    rng = np.random.RandomState(seed)

    # Hand-tuned profiles for key countries (resources, labour_abundance,
    # labour_skill, capital_stock, infrastructure, tech_level)
    MANUAL_PROFILES = {
        "USA": [0.55, 0.25, 0.92, 0.95, 0.93, 0.95],
        "CHN": [0.50, 0.85, 0.65, 0.80, 0.82, 0.78],
        "JPN": [0.10, 0.20, 0.90, 0.92, 0.95, 0.93],
        "DEU": [0.15, 0.18, 0.90, 0.93, 0.95, 0.92],
        "GBR": [0.25, 0.20, 0.88, 0.85, 0.90, 0.88],
        "IND": [0.45, 0.92, 0.45, 0.42, 0.45, 0.48],
        "FRA": [0.20, 0.20, 0.85, 0.85, 0.90, 0.85],
        "BRA": [0.75, 0.72, 0.40, 0.48, 0.50, 0.42],
        "CAN": [0.75, 0.15, 0.85, 0.82, 0.88, 0.82],
        "RUS": [0.85, 0.55, 0.65, 0.60, 0.55, 0.60],
        "KOR": [0.08, 0.25, 0.88, 0.88, 0.92, 0.90],
        "AUS": [0.80, 0.12, 0.82, 0.78, 0.85, 0.78],
        "MEX": [0.55, 0.78, 0.42, 0.52, 0.55, 0.40],
        "SAU": [0.92, 0.55, 0.45, 0.55, 0.65, 0.35],
        "IDN": [0.65, 0.88, 0.35, 0.38, 0.42, 0.30],
        "NLD": [0.20, 0.12, 0.88, 0.88, 0.95, 0.88],
        "CHE": [0.05, 0.08, 0.95, 0.92, 0.95, 0.95],
        "TWN": [0.05, 0.22, 0.88, 0.85, 0.90, 0.92],
        "SGP": [0.02, 0.30, 0.90, 0.88, 0.98, 0.88],
        "VNM": [0.40, 0.90, 0.40, 0.42, 0.50, 0.32],
        "THA": [0.45, 0.78, 0.48, 0.55, 0.60, 0.42],
        "ARE": [0.80, 0.65, 0.55, 0.65, 0.85, 0.45],
        "NGA": [0.70, 0.85, 0.22, 0.18, 0.20, 0.12],
        "ZAF": [0.72, 0.72, 0.42, 0.45, 0.50, 0.38],
        "BGD": [0.20, 0.95, 0.25, 0.25, 0.30, 0.15],
        "POL": [0.25, 0.35, 0.72, 0.68, 0.75, 0.62],
        "TUR": [0.35, 0.62, 0.55, 0.58, 0.62, 0.48],
        "SWE": [0.30, 0.10, 0.90, 0.88, 0.92, 0.92],
        "NOR": [0.82, 0.08, 0.88, 0.85, 0.90, 0.85],
        "IRQ": [0.85, 0.68, 0.25, 0.20, 0.25, 0.12],
        "KWT": [0.88, 0.55, 0.48, 0.52, 0.68, 0.30],
        "QAT": [0.85, 0.60, 0.50, 0.55, 0.78, 0.35],
        "CHL": [0.72, 0.42, 0.52, 0.50, 0.62, 0.42],
        "DZA": [0.78, 0.62, 0.35, 0.30, 0.35, 0.20],
        "LBY": [0.82, 0.52, 0.30, 0.22, 0.25, 0.12],
        "ETH": [0.35, 0.92, 0.15, 0.10, 0.15, 0.08],
        "KHM": [0.25, 0.92, 0.22, 0.22, 0.30, 0.12],
        "MYS": [0.55, 0.62, 0.58, 0.62, 0.72, 0.55],
        "PHL": [0.30, 0.85, 0.45, 0.35, 0.40, 0.30],
        "EGY": [0.40, 0.78, 0.38, 0.35, 0.42, 0.25],
        "HKG": [0.02, 0.22, 0.85, 0.82, 0.95, 0.78],
        "BEL": [0.08, 0.15, 0.85, 0.85, 0.92, 0.82],
        "IRL": [0.08, 0.12, 0.88, 0.82, 0.88, 0.85],
        "ISR": [0.10, 0.18, 0.90, 0.82, 0.85, 0.92],
        "AZE": [0.75, 0.55, 0.42, 0.35, 0.40, 0.22],
        "UKR": [0.45, 0.55, 0.62, 0.45, 0.45, 0.45],
        "BLR": [0.20, 0.42, 0.62, 0.50, 0.52, 0.42],
        "COD": [0.85, 0.82, 0.10, 0.05, 0.08, 0.03],
    }

    records = []
    for iso, (name, pop, gdp, lfpr, ind_idx) in countries.items():
        if iso in MANUAL_PROFILES:
            prof = MANUAL_PROFILES[iso]
        else:
            # Heuristic generation
            gdp_pc = gdp / max(pop, 0.1)  # GDP per capita in thousands

            # Natural resources: inversely correlated with GDP/cap for
            # non-rich countries, but some rich countries are resource-rich
            resources = np.clip(
                0.3 + 0.3 * rng.randn() + 0.1 * (1.0 - min(gdp_pc / 60, 1.0)),
                0.05, 0.95
            )

            # Labour abundance: inversely proportional to GDP per capita
            labour_abund = np.clip(
                0.9 - 0.7 * min(gdp_pc / 60, 1.0) + 0.1 * rng.randn(),
                0.05, 0.95
            )

            # Labour skill: correlated with industrialization and GDP/cap
            labour_skill = np.clip(
                0.5 * ind_idx + 0.3 * min(gdp_pc / 60, 1.0) + 0.1 * rng.randn(),
                0.05, 0.95
            )

            # Capital stock
            capital = np.clip(
                0.4 * ind_idx + 0.4 * min(gdp_pc / 60, 1.0) + 0.1 * rng.randn(),
                0.05, 0.95
            )

            # Infrastructure
            infra = np.clip(
                0.3 * ind_idx + 0.5 * min(gdp_pc / 40, 1.0) + 0.1 * rng.randn(),
                0.05, 0.95
            )

            # Tech level
            tech = np.clip(
                0.5 * ind_idx + 0.3 * min(gdp_pc / 70, 1.0) + 0.1 * rng.randn(),
                0.03, 0.95
            )

            prof = [resources, labour_abund, labour_skill, capital, infra, tech]

        records.append({
            "iso": iso,
            "name": name,
            "population": pop,
            "gdp": gdp,
            "lfpr": lfpr,
            "industrialization": countries[iso][4],
            "natural_resources": prof[0],
            "labour_abundance": prof[1],
            "labour_skill": prof[2],
            "capital_stock": prof[3],
            "infrastructure": prof[4],
            "tech_level": prof[5],
        })

    return pd.DataFrame(records).set_index("iso")


# ============================================================
# 4. CORE LTV TRADE MODEL
# ============================================================

class LTVTradeModel:
    """
    Labour Theory of Value Trade Model

    Key theoretical components:

    1. VALUE DETERMINATION:
       - Value of a commodity = c + v + s
         where c = constant capital (raw materials, depreciation)
               v = variable capital (wages = socially necessary labour time)
               s = surplus value
       - The organic composition of capital q = c / (c + v) determines
         how capital-intensive production is

    2. COMPARATIVE ADVANTAGE (LTV version):
       - Countries export goods where their labour is relatively more
         productive (lower SNLT per unit of use-value)
       - Productivity depends on: technology, skill, capital stock,
         resource endowments
       - "Unequal exchange": countries with lower organic composition
         transfer value to those with higher composition through trade

    3. TRADE FLOWS:
       - Supply: country's production capacity in each commodity
       - Demand: derived from GDP, population, industrial structure
       - Net exports = supply - domestic_demand
       - Trade is balanced iteratively across all country pairs
    """

    def __init__(self, countries: dict, seed: int = 42):
        self.rng = np.random.RandomState(seed)
        self.countries = countries
        self.endowments = build_country_endowments(countries, seed)
        self.commodities = COMMODITY_GROUPS
        self.years = list(range(2019, 2024))

        # Compute labour productivity matrix
        self.productivity = self._compute_productivity()

        # Compute production capacity
        self.capacity = self._compute_production_capacity()

    def _compute_productivity(self) -> pd.DataFrame:
        """
        Compute each country's labour productivity for each commodity.

        Productivity_i_j = f(endowments_i, commodity_requirements_j)

        Higher productivity = lower SNLT = lower value per unit =
        competitive advantage
        """
        records = []

        for iso in self.endowments.index:
            e = self.endowments.loc[iso]
            for cid, comm in self.commodities.items():
                # Productivity is a weighted combination of how well
                # the country's endowments match the commodity's requirements

                # Resource match
                resource_match = (
                    e["natural_resources"] * comm.resource_dependence +
                    (1 - comm.resource_dependence) * 0.5
                )

                # Labour match (abundant cheap labour helps for high-SNLT goods)
                labour_cost_advantage = (
                    e["labour_abundance"] * (1 - comm.skill_intensity) *
                    (comm.base_snlt / 15.0)  # Normalize by max SNLT
                )

                # Skill match
                skill_match = e["labour_skill"] * comm.skill_intensity

                # Capital match
                capital_match = e["capital_stock"] * comm.capital_intensity

                # Tech match
                tech_match = e["tech_level"] * (comm.skill_intensity * 0.5 +
                                                 comm.capital_intensity * 0.5)

                # Infrastructure (affects ability to trade bulky goods)
                infra_effect = (
                    e["infrastructure"] * (0.3 + 0.7 * comm.bulk_weight)
                )

                # Combined productivity score
                productivity = (
                    0.20 * resource_match +
                    0.15 * labour_cost_advantage +
                    0.20 * skill_match +
                    0.20 * capital_match +
                    0.15 * tech_match +
                    0.10 * infra_effect
                )

                # Add some noise
                productivity *= (1 + 0.05 * self.rng.randn())
                productivity = max(productivity, 0.01)

                records.append({
                    "iso": iso,
                    "commodity": cid,
                    "productivity": productivity
                })

        df = pd.DataFrame(records)
        return df.pivot(index="iso", columns="commodity", values="productivity")

    def _compute_production_capacity(self) -> pd.DataFrame:
        """
        Each country's total production capacity is proportional to its
        GDP and labour force. The distribution across commodities follows
        the productivity profile (countries allocate labour to their most
        productive sectors).
        """
        capacity = pd.DataFrame(
            index=self.endowments.index,
            columns=list(self.commodities.keys()),
            dtype=float
        )

        for iso in self.endowments.index:
            e = self.endowments.loc[iso]
            total_labour_value = (
                e["population"] * e["lfpr"] *
                (0.3 + 0.7 * e["labour_skill"])  # Effective labour units
            )

            # Allocate across sectors proportional to productivity
            prod_scores = self.productivity.loc[iso].values
            # Raise to power to concentrate in best sectors
            allocation = prod_scores ** 2
            allocation = allocation / allocation.sum()

            # Scale by GDP to get absolute capacity
            # GDP is in billions; capacity represents potential export value
            total_capacity = e["gdp"] * 0.15  # ~15% of GDP is tradeable goods

            capacity.loc[iso] = allocation * total_capacity

        return capacity

    def _compute_domestic_demand(self, year: int) -> pd.DataFrame:
        """
        Domestic demand for each commodity, based on:
        - GDP (income effect)
        - Population (basic needs)
        - Industrialization level (demand for capital goods)
        - Development stage (Engel's law: food share decreases with income)
        """
        demand = pd.DataFrame(
            index=self.endowments.index,
            columns=list(self.commodities.keys()),
            dtype=float
        )

        year_idx = year - 2019

        for iso in self.endowments.index:
            e = self.endowments.loc[iso]
            gdp_pc = e["gdp"] / max(e["population"], 0.1)

            for cid, comm in self.commodities.items():
                # Base demand proportional to GDP
                base = e["gdp"] * 0.12  # Total import demand ~12% of GDP

                # Commodity-specific demand shares
                if cid in ("AGR",):
                    # Food: higher share for poor countries (Engel's law)
                    share = 0.15 * (1.5 - min(gdp_pc / 60, 1.0))
                elif cid in ("OIL", "GAS"):
                    # Energy: universal need, scales with industrialization
                    share = 0.12 * (0.5 + 0.5 * e["industrialization"])
                elif cid in ("ELE", "MAC"):
                    # Capital goods: higher for industrializing countries
                    share = 0.10 * (0.3 + 0.7 * e["industrialization"])
                elif cid in ("PHA", "MED"):
                    # Healthcare: scales with income
                    share = 0.04 * min(gdp_pc / 30, 1.5)
                elif cid in ("AUT",):
                    share = 0.08 * min(gdp_pc / 20, 1.2)
                elif cid in ("TEX",):
                    share = 0.06 * (1.3 - 0.3 * min(gdp_pc / 50, 1.0))
                elif cid in ("ARM",):
                    share = 0.01 * (0.5 + 0.5 * self.rng.rand())
                elif cid in ("AIR",):
                    share = 0.02 * min(gdp_pc / 40, 1.0)
                else:
                    # Default: proportional to global trade share
                    share = comm.global_demand_billions / sum(
                        c.global_demand_billions for c in self.commodities.values()
                    )
                    share *= 0.8  # Scale down

                # Year effects
                growth_factor = (1 + comm.growth_trend) ** year_idx

                # COVID shock in 2020
                if year == 2020:
                    if cid in ("OIL", "GAS", "AUT", "AIR", "TEX"):
                        growth_factor *= 0.82
                    elif cid in ("PHA", "MED"):
                        growth_factor *= 1.15
                    else:
                        growth_factor *= 0.92

                # 2021 recovery
                if year == 2021:
                    growth_factor *= 1.08

                # 2022 energy price spike
                if year == 2022:
                    if cid in ("OIL", "GAS"):
                        growth_factor *= 1.45
                    elif cid in ("FRT", "CHM"):
                        growth_factor *= 1.20
                    elif cid in ("AGR",):
                        growth_factor *= 1.15

                # 2023 normalization
                if year == 2023:
                    if cid in ("OIL", "GAS"):
                        growth_factor *= 0.88

                demand.loc[iso, cid] = base * share * growth_factor

        return demand

    def _compute_labour_values(self, year: int) -> pd.DataFrame:
        """
        Compute the labour value (in abstract labour time) of each
        commodity in each country.

        Value = c + v + s where:
        - c (constant capital) = capital_intensity * base cost
        - v (variable capital) = SNLT * wage rate
        - s (surplus value) = rate_of_exploitation * v

        Countries with higher productivity have lower values per unit
        → they can undersell others → export advantage
        """
        values = pd.DataFrame(
            index=self.endowments.index,
            columns=list(self.commodities.keys()),
            dtype=float
        )

        for iso in self.endowments.index:
            e = self.endowments.loc[iso]

            # Country-specific wage rate (proportional to productivity)
            wage_rate = (
                0.2 + 0.8 * (e["gdp"] / max(e["population"], 0.1)) / 80.0
            )
            wage_rate = min(wage_rate, 1.0)

            # Rate of surplus extraction (higher in developing countries)
            exploitation_rate = 1.5 - 0.5 * wage_rate

            for cid, comm in self.commodities.items():
                prod = self.productivity.loc[iso, cid]

                # SNLT for this country-commodity pair
                snlt = comm.base_snlt / max(prod, 0.01)

                # Value decomposition
                v = snlt * wage_rate  # Variable capital
                c = v * comm.capital_intensity / (1 - comm.capital_intensity + 0.01)
                s = exploitation_rate * v

                total_value = c + v + s
                values.loc[iso, cid] = total_value

        return values

    def generate_trade_data(self) -> pd.DataFrame:
        """
        Main method: generate import/export data for all countries,
        all commodities, all years.

        Algorithm:
        1. For each year, compute production capacity, domestic demand,
           and labour values
        2. Net trade position = capacity - demand (adjusted for
           competitiveness)
        3. Positive net = export surplus; negative = import need
        4. Scale to match realistic global trade volumes
        5. Add noise for realism
        """
        all_records = []

        for year in self.years:
            print(f"Generating trade data for {year}...")

            demand = self._compute_domestic_demand(year)
            values = self._compute_labour_values(year)

            year_idx = year - 2019

            # Year-specific capacity adjustment
            capacity_adj = self.capacity.copy()

            # COVID production shock
            if year == 2020:
                capacity_adj *= 0.90
            elif year == 2021:
                capacity_adj *= 1.05
            elif year == 2022:
                capacity_adj *= 1.08
            elif year == 2023:
                capacity_adj *= 1.06

            for iso in self.endowments.index:
                for cid, comm in self.commodities.items():
                    cap = capacity_adj.loc[iso, cid]
                    dem = demand.loc[iso, cid]
                    val = values.loc[iso, cid]
                    prod = self.productivity.loc[iso, cid]

                    # Competitiveness: lower value = more competitive
                    # Compare to global average
                    avg_value = values[cid].mean()
                    competitiveness = avg_value / max(val, 0.001)
                    competitiveness = np.clip(competitiveness, 0.3, 3.0)

                    # Export potential: capacity * competitiveness
                    export_potential = cap * competitiveness

                    # Trade balance for this commodity
                    net_trade = export_potential - dem

                    # Convert to realistic dollar values
                    # Scale factor: match global trade volumes
                    global_total = comm.global_demand_billions
                    n_countries = len(self.endowments)

                    # Country's share of world GDP
                    world_gdp = self.endowments["gdp"].sum()
                    gdp_share = self.endowments.loc[iso, "gdp"] / world_gdp

                    # Scale exports and imports
                    scale = global_total * 0.5  # Each good is traded ~50% of total

                    if net_trade > 0:
                        # Net exporter
                        exports = scale * gdp_share * competitiveness * (
                            1 + 0.5 * prod
                        )
                        imports = max(exports - abs(net_trade) * gdp_share * 50,
                                     exports * 0.1)
                    else:
                        # Net importer
                        imports = scale * gdp_share * (
                            1.2 - 0.2 * competitiveness
                        ) * (1 + 0.3 * (1 - prod))
                        exports = max(imports - abs(net_trade) * gdp_share * 50,
                                     imports * 0.05)

                    # Ensure non-negative
                    exports = max(exports, 0.001)
                    imports = max(imports, 0.001)

                    # Apply year-over-year growth trend
                    growth = (1 + comm.growth_trend) ** year_idx
                    exports *= growth
                    imports *= growth

                    # Add realistic noise (±5-15%)
                    noise_exp = 1 + 0.08 * self.rng.randn()
                    noise_imp = 1 + 0.08 * self.rng.randn()
                    exports *= max(noise_exp, 0.5)
                    imports *= max(noise_imp, 0.5)

                    # Country-specific shocks
                    if iso == "RUS" and year >= 2022:
                        # Sanctions effect
                        if cid not in ("OIL", "GAS", "MIN"):
                            exports *= 0.60
                            imports *= 0.55
                        else:
                            # Redirect but still export energy
                            exports *= 0.85

                    if iso == "UKR" and year >= 2022:
                        exports *= 0.45
                        imports *= 0.55

                    all_records.append({
                        "year": year,
                        "country_iso": iso,
                        "country_name": self.endowments.loc[iso, "name"],
                        "commodity_code": cid,
                        "commodity_name": comm.name,
                        "exports_billions_usd": round(exports, 4),
                        "imports_billions_usd": round(imports, 4),
                        "net_exports_billions_usd": round(exports - imports, 4),
                        "labour_value_index": round(val, 4),
                        "competitiveness_index": round(competitiveness, 4),
                    })

        df = pd.DataFrame(all_records)

        # Post-processing: normalize to match realistic global totals
        df = self._normalize_global_totals(df)

        return df

    def _normalize_global_totals(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Rescale so that global totals per commodity roughly match
        the specified global_demand_billions targets.
        """
        for year in self.years:
            for cid, comm in self.commodities.items():
                mask = (df["year"] == year) & (df["commodity_code"] == cid)

                current_total_exp = df.loc[mask, "exports_billions_usd"].sum()
                target = comm.global_demand_billions

                # Apply year-specific target adjustment
                year_factor = 1.0
                if year == 2020:
                    year_factor = 0.88
                elif year == 2021:
                    year_factor = 1.10
                elif year == 2022:
                    year_factor = 1.18
                elif year == 2023:
                    year_factor = 1.12

                adj_target = target * year_factor * (
                    (1 + comm.growth_trend) ** (year - 2019)
                )

                if current_total_exp > 0:
                    scale = adj_target / current_total_exp
                    df.loc[mask, "exports_billions_usd"] *= scale
                    df.loc[mask, "imports_billions_usd"] *= scale
                    df.loc[mask, "net_exports_billions_usd"] = (
                        df.loc[mask, "exports_billions_usd"] -
                        df.loc[mask, "imports_billions_usd"]
                    )

        # Round final values
        for col in ["exports_billions_usd", "imports_billions_usd",
                     "net_exports_billions_usd"]:
            df[col] = df[col].round(3)

        return df


# ============================================================
# 5. ANALYSIS & SUMMARY TOOLS
# ============================================================

def compute_country_summaries(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate trade by country and year."""
    summary = df.groupby(["year", "country_iso", "country_name"]).agg(
        total_exports=("exports_billions_usd", "sum"),
        total_imports=("imports_billions_usd", "sum"),
    ).reset_index()

    summary["trade_balance"] = summary["total_exports"] - summary["total_imports"]
    summary["trade_volume"] = summary["total_exports"] + summary["total_imports"]
    summary["trade_openness"] = summary["trade_volume"]  # Could divide by GDP

    return summary.round(2)


def compute_commodity_summaries(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate trade by commodity and year."""
    summary = df.groupby(["year", "commodity_code", "commodity_name"]).agg(
        global_exports=("exports_billions_usd", "sum"),
        global_imports=("imports_billions_usd", "sum"),
        top_exporter_value=("exports_billions_usd", "max"),
    ).reset_index()

    return summary.round(2)


def compute_unequal_exchange(df: pd.DataFrame,
                              endowments: pd.DataFrame) -> pd.DataFrame:
    """
    Estimate unequal exchange: the transfer of value from
    low-wage to high-wage countries through trade.

    When a low-wage country exports goods with high embodied labour
    in exchange for goods from high-wage countries with lower embodied
    labour but equal price, value is transferred.
    """
    results = []

    for year in df["year"].unique():
        year_data = df[df["year"] == year]

        for iso in endowments.index:
            country_data = year_data[year_data["country_iso"] == iso]

            gdp_pc = endowments.loc[iso, "gdp"] / max(
                endowments.loc[iso, "population"], 0.1
            )

            # Wage index (0-1)
            wage_idx = min(gdp_pc / 80.0, 1.0)

            # Value transferred = exports * (1 - wage_ratio_to_world_avg)
            world_avg_wage = 0.35  # Approximate

            total_exports = country_data["exports_billions_usd"].sum()
            total_imports = country_data["imports_billions_usd"].sum()

            # Simplified unequal exchange estimate
            value_transfer = total_exports * (world_avg_wage - wage_idx)

            results.append({
                "year": year,
                "country_iso": iso,
                "country_name": endowments.loc[iso, "name"],
                "wage_index": round(wage_idx, 3),
                "total_exports": round(total_exports, 2),
                "total_imports": round(total_imports, 2),
                "estimated_value_transfer": round(value_transfer, 2),
                "transfer_direction": (
                    "outflow (exploited)" if value_transfer > 0
                    else "inflow (beneficiary)"
                ),
            })

    return pd.DataFrame(results)


# ============================================================
# 6. MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    print("=" * 70)
    print("LABOUR THEORY OF VALUE — GLOBAL TRADE MODEL")
    print("=" * 70)

    # Initialize model
    model = LTVTradeModel(COUNTRIES, seed=42)

    # Generate full trade dataset
    trade_data = model.generate_trade_data()

    print(f"\nGenerated {len(trade_data):,} trade records")
    print(f"  Countries: {trade_data['country_iso'].nunique()}")
    print(f"  Commodities: {trade_data['commodity_code'].nunique()}")
    print(f"  Years: {sorted(trade_data['year'].unique())}")

    # Country summaries
    country_summary = compute_country_summaries(trade_data)

    print("\n" + "=" * 70)
    print("TOP 15 EXPORTERS (2023)")
    print("=" * 70)
    top_2023 = country_summary[country_summary["year"] == 2023].nlargest(
        15, "total_exports"
    )
    print(top_2023[["country_name", "total_exports", "total_imports",
                     "trade_balance"]].to_string(index=False))

    print("\n" + "=" * 70)
    print("LARGEST TRADE DEFICITS (2023)")
    print("=" * 70)
    deficits = country_summary[country_summary["year"] == 2023].nsmallest(
        10, "trade_balance"
    )
    print(deficits[["country_name", "total_exports", "total_imports",
                     "trade_balance"]].to_string(index=False))

    print("\n" + "=" * 70)
    print("LARGEST TRADE SURPLUSES (2023)")
    print("=" * 70)
    surpluses = country_summary[country_summary["year"] == 2023].nlargest(
        10, "trade_balance"
    )
    print(surpluses[["country_name", "total_exports", "total_imports",
                      "trade_balance"]].to_string(index=False))

    # Commodity summaries
    commodity_summary = compute_commodity_summaries(trade_data)
    print("\n" + "=" * 70)
    print("GLOBAL TRADE BY COMMODITY (2023)")
    print("=" * 70)
    comm_2023 = commodity_summary[commodity_summary["year"] == 2023].sort_values(
        "global_exports", ascending=False
    )
    print(comm_2023[["commodity_name", "global_exports"]].to_string(index=False))

    # Unequal exchange analysis
    ue = compute_unequal_exchange(trade_data, model.endowments)
    print("\n" + "=" * 70)
    print("UNEQUAL EXCHANGE ESTIMATES (2023)")
    print("=" * 70)
    ue_2023 = ue[ue["year"] == 2023].sort_values("estimated_value_transfer")
    print("\nLargest value OUTFLOWS (value transferred away):")
    print(ue_2023.nlargest(10, "estimated_value_transfer")[
        ["country_name", "wage_index", "total_exports",
         "estimated_value_transfer"]
    ].to_string(index=False))

    print("\nLargest value INFLOWS (value received via unequal exchange):")
    print(ue_2023.nsmallest(10, "estimated_value_transfer")[
        ["country_name", "wage_index", "total_exports",
         "estimated_value_transfer"]
    ].to_string(index=False))

    # Save to CSV
    trade_data.to_csv("ltv_trade_data_full.csv", index=False)
    country_summary.to_csv("ltv_country_summaries.csv", index=False)
    commodity_summary.to_csv("ltv_commodity_summaries.csv", index=False)
    ue.to_csv("ltv_unequal_exchange.csv", index=False)

    print("\n" + "=" * 70)
    print("Data saved to CSV files:")
    print("  - ltv_trade_data_full.csv (detailed by country × commodity × year)")
    print("  - ltv_country_summaries.csv (country totals by year)")
    print("  - ltv_commodity_summaries.csv (commodity totals by year)")
    print("  - ltv_unequal_exchange.csv (value transfer estimates)")
    print("=" * 70)

    # === EXAMPLE: Drill down into a specific country ===
    print("\n" + "=" * 70)
    print("EXAMPLE: UNITED STATES TRADE PROFILE (2023)")
    print("=" * 70)
    us_2023 = trade_data[
        (trade_data["country_iso"] == "USA") & (trade_data["year"] == 2023)
    ].sort_values("exports_billions_usd", ascending=False)

    print(us_2023[["commodity_name", "exports_billions_usd",
                    "imports_billions_usd", "net_exports_billions_usd",
                    "competitiveness_index"]].to_string(index=False))

    # === FIVE-YEAR TREND for USA ===
    print("\n" + "=" * 70)
    print("USA TRADE BALANCE OVER TIME")
    print("=" * 70)
    us_trend = country_summary[country_summary["country_iso"] == "USA"]
    print(us_trend[["year", "total_exports", "total_imports",
                     "trade_balance"]].to_string(index=False))

<>:194: SyntaxWarning: invalid escape sequence '\$'
<>:194: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1970097349.py:194: SyntaxWarning: invalid escape sequence '\$'
  - base_snlt: Base socially necessary labour time index (hours per \$1000 of value)


LABOUR THEORY OF VALUE — GLOBAL TRADE MODEL
Generating trade data for 2019...
Generating trade data for 2020...
Generating trade data for 2021...
Generating trade data for 2022...
Generating trade data for 2023...

Generated 17,595 trade records
  Countries: 153
  Commodities: 23
  Years: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]

TOP 15 EXPORTERS (2023)
  country_name  total_exports  total_imports  trade_balance
         China        7621.32        4679.04        2942.28
 United States        1800.29        9295.11       -7494.83
         India        1573.79        1121.35         452.43
        Brazil         752.42         692.35          60.07
         Japan         668.25        1463.89        -795.64
     Indonesia         541.78         503.03          38.75
       Germany         527.62        1522.14        -994.52
        Russia         515.17         455.33          59.84
        Mexico         485.68         505.75         -20.07
Unit

In [2]:
"""
Industrialization & De-industrialization Dynamics Module
=========================================================

Extends the LTV Trade Model with dynamic structural change:

THEORETICAL FRAMEWORK:
======================

1. INTELLIGENT INDUSTRIALIZATION (II):
   - Countries strategically upgrade their productive capacity
   - Move up the value chain: AGR/TEX → STL/CHM → MAC/ELE → PHA/AIR
   - Requires: state investment, technology absorption, skill formation
   - Historical examples: South Korea, China, Taiwan, Germany

2. PREMATURE DE-INDUSTRIALIZATION (PDI):
   - Countries lose manufacturing capacity before reaching high income
   - Driven by: persistent trade deficits, import dependence,
     Dutch disease, structural adjustment
   - Manufacturing share of GDP peaks at lower income levels

3. THE SERVICES TRAP:
   - Deficit countries shift labour from tradeable goods → services
   - Services are harder to export & have lower productivity growth
   - Creates structural dependency: must import manufactured goods
   - Financed by: debt, asset sales, remittances, or resource rents
   - This module models the ENTRY CONDITIONS for the trap
   - The rug pull mechanism (sudden financing crisis) is deferred

KEY DYNAMICS:
=============
- Cumulative trade deficits erode industrial capacity
- Cumulative surpluses enable reinvestment → capacity expansion
- Industrialization requires crossing threshold investment levels
- "Hysteresis": once capacity is lost, it's very hard to rebuild
- Path dependence: early industrializers have compounding advantages
"""

import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional
from enum import Enum
import warnings

# Import base model (assuming previous code is in ltv_trade_model.py)
# from ltv_trade_model import *

# If running standalone, we need the base model components
# (paste the COUNTRIES, COMMODITY_GROUPS, etc. from previous module or import)


# ============================================================
# 1. STRUCTURAL CLASSIFICATION
# ============================================================

class IndustrialStage(Enum):
    """
    Stages of industrial development, roughly following
    the Rostow/Gerschenkron/East Asian development model
    but grounded in LTV production relations.
    """
    PRIMARY_EXPORT = "primary_export"          # Raw materials & agriculture
    LIGHT_MANUFACTURING = "light_manufacturing" # Textiles, simple assembly
    HEAVY_INDUSTRY = "heavy_industry"           # Steel, chemicals, machinery
    ADVANCED_MANUFACTURING = "advanced_mfg"     # Electronics, autos, pharma
    FRONTIER_TECH = "frontier_tech"             # Aerospace, semiconductors, AI hardware

    # De-industrialization paths
    PREMATURE_DEINDUSTRIAL = "premature_deindustrial"  # Lost mfg before maturity
    SERVICES_DEPENDENT = "services_dependent"           # Trapped in service economy
    RESOURCE_DEPENDENT = "resource_dependent"           # Dutch disease path


class TradePosture(Enum):
    """A country's structural trade position."""
    STRATEGIC_SURPLUS = "strategic_surplus"        # Intentional export-led growth
    BALANCED_INDUSTRIAL = "balanced_industrial"    # Roughly balanced, strong mfg
    DEFICIT_INDUSTRIAL = "deficit_industrial"      # Deficit but still has mfg base
    DEFICIT_ERODING = "deficit_eroding"            # Deficit & losing mfg capacity
    DEFICIT_DEPENDENT = "deficit_dependent"        # Structural deficit, weak mfg
    RESOURCE_SURPLUS = "resource_surplus"          # Surplus from commodities only


# ============================================================
# 2. COMMODITY VALUE CHAIN TIERS
# ============================================================

# Organize commodities into value-chain tiers
# Higher tier = more value-added, more skill/capital intensive
VALUE_CHAIN_TIERS = {
    0: ["AGR"],                              # Primary
    1: ["OIL", "GAS", "MIN", "WDP"],        # Resource extraction
    2: ["TEX", "FUR", "TOY", "CER"],        # Light manufacturing
    3: ["STL", "PLR", "CHM", "FRT"],        # Basic heavy industry
    4: ["AUT", "MAC", "SHP"],               # Complex manufacturing
    5: ["ELE", "OPT", "MED"],               # Advanced manufacturing
    6: ["PHA", "AIR", "ARM"],               # Frontier/strategic
    7: ["GEM"],                              # Special (resource + skill)
}

# Reverse lookup
COMMODITY_TO_TIER = {}
for tier, codes in VALUE_CHAIN_TIERS.items():
    for code in codes:
        COMMODITY_TO_TIER[code] = tier

# Strategic commodity groups for industrialization policy
STRATEGIC_SECTORS = {
    "import_substitution_basic": ["TEX", "FUR", "CER", "AGR"],
    "import_substitution_intermediate": ["STL", "PLR", "CHM"],
    "export_promotion_light": ["TEX", "TOY", "FUR"],
    "export_promotion_heavy": ["AUT", "MAC", "STL"],
    "technology_acquisition": ["ELE", "OPT", "MED", "MAC"],
    "strategic_independence": ["PHA", "ARM", "AIR", "ELE"],
}


# ============================================================
# 3. COUNTRY STRUCTURAL STATE
# ============================================================

@dataclass
class CountryStructuralState:
    """
    Tracks the evolving industrial structure of a country.
    This is the core state object that gets updated each period.
    """
    iso: str
    name: str

    # Current classification
    industrial_stage: IndustrialStage = IndustrialStage.PRIMARY_EXPORT
    trade_posture: TradePosture = TradePosture.BALANCED_INDUSTRIAL

    # Capacity by commodity (evolving over time)
    # Maps commodity_code -> capacity_index (0-100)
    sector_capacity: Dict[str, float] = field(default_factory=dict)

    # Cumulative metrics
    cumulative_trade_balance: float = 0.0  # Running sum of trade balances
    cumulative_mfg_investment: float = 0.0  # Accumulated industrial investment
    cumulative_capacity_erosion: float = 0.0  # Lost capacity from deficits

    # Structural indicators
    manufacturing_share_gdp: float = 0.0  # % of GDP from manufacturing
    services_share_gdp: float = 0.0       # % of GDP from services
    primary_share_gdp: float = 0.0        # % of GDP from primary sector

    # Industrialization policy
    has_industrial_policy: bool = False
    policy_target_sectors: List[str] = field(default_factory=list)
    policy_effectiveness: float = 0.0  # 0-1 scale

    # Services trap indicators
    services_trap_risk: float = 0.0  # 0-1 probability of entering trap
    years_in_deficit: int = 0
    import_dependence_index: float = 0.0  # How dependent on imported mfg goods

    # History
    history: Dict[int, dict] = field(default_factory=dict)


# ============================================================
# 4. INDUSTRIALIZATION ENGINE
# ============================================================

class IndustrializationEngine:
    """
    Models the dynamic process of industrialization and
    de-industrialization based on trade patterns, investment
    decisions, and structural change.

    Core mechanism (LTV grounding):
    - Trade SURPLUSES in manufactured goods → realized surplus value
      → can be reinvested in expanding constant capital (machinery,
      factories) → increased organic composition → higher productivity
      → stronger competitive position → virtuous cycle

    - Trade DEFICITS in manufactured goods → value leakage → reduced
      surplus available for investment → capacity stagnation/decline
      → lower competitiveness → deeper deficits → vicious cycle

    This is the fundamental asymmetry that drives divergence between
    industrializing and de-industrializing countries.
    """

    def __init__(self, trade_data: pd.DataFrame, endowments: pd.DataFrame,
                 seed: int = 42):
        self.rng = np.random.RandomState(seed)
        self.trade_data = trade_data.copy()
        self.endowments = endowments.copy()
        self.years = sorted(trade_data["year"].unique())
        self.countries = endowments.index.tolist()
        self.commodity_codes = sorted(trade_data["commodity_code"].unique())

        # Initialize structural states
        self.states: Dict[str, CountryStructuralState] = {}
        self._initialize_states()

        # Parameters
        self.params = IndustrializationParams()

    def _initialize_states(self):
        """Set initial structural state for each country based on endowments."""
        for iso in self.countries:
            e = self.endowments.loc[iso]

            state = CountryStructuralState(
                iso=iso,
                name=e["name"],
            )

            # Initialize sector capacities from endowments
            gdp = e["gdp"]
            ind = e["industrialization"]
            tech = e["tech_level"]
            resources = e["natural_resources"]
            skill = e["labour_skill"]

            for code in self.commodity_codes:
                tier = COMMODITY_TO_TIER.get(code, 3)

                # Base capacity depends on industrialization level and tier
                if tier <= 1:
                    # Primary/resource: depends on resource endowment
                    cap = 30 + 60 * resources + 10 * self.rng.rand()
                elif tier <= 2:
                    # Light manufacturing: accessible with basic industry
                    cap = 20 + 40 * ind + 20 * (1 - skill) * e["labour_abundance"]
                elif tier <= 3:
                    # Heavy industry: needs capital and some skill
                    cap = 10 + 50 * ind + 20 * e["capital_stock"]
                elif tier <= 4:
                    # Complex manufacturing
                    cap = 5 + 40 * ind + 30 * skill + 15 * tech
                elif tier <= 5:
                    # Advanced manufacturing
                    cap = 2 + 30 * tech + 30 * skill + 20 * e["capital_stock"]
                else:
                    # Frontier
                    cap = 1 + 40 * tech + 30 * skill + 10 * ind

                cap += 5 * self.rng.randn()
                state.sector_capacity[code] = np.clip(cap, 1.0, 100.0)

            # Determine initial industrial stage
            state.industrial_stage = self._classify_stage(state, e)

            # Initial GDP composition
            state.manufacturing_share_gdp = np.clip(
                0.05 + 0.25 * ind + 0.05 * self.rng.randn(), 0.03, 0.35
            )
            state.primary_share_gdp = np.clip(
                0.30 - 0.25 * ind + 0.05 * resources + 0.03 * self.rng.randn(),
                0.02, 0.55
            )
            state.services_share_gdp = 1.0 - state.manufacturing_share_gdp - state.primary_share_gdp

            # Industrial policy (heuristic: some countries have it)
            if iso in ("CHN", "KOR", "TWN", "SGP", "VNM", "IND", "MYS",
                       "THA", "IDN", "TUR", "ETH", "RWA", "BGD"):
                state.has_industrial_policy = True
                state.policy_effectiveness = np.clip(
                    0.3 + 0.4 * tech + 0.2 * e["capital_stock"] + 0.1 * self.rng.rand(),
                    0.1, 0.9
                )
                # Assign target sectors based on current stage
                stage = state.industrial_stage
                if stage == IndustrialStage.PRIMARY_EXPORT:
                    state.policy_target_sectors = STRATEGIC_SECTORS["import_substitution_basic"]
                elif stage == IndustrialStage.LIGHT_MANUFACTURING:
                    state.policy_target_sectors = STRATEGIC_SECTORS["export_promotion_light"]
                elif stage == IndustrialStage.HEAVY_INDUSTRY:
                    state.policy_target_sectors = STRATEGIC_SECTORS["export_promotion_heavy"]
                elif stage in (IndustrialStage.ADVANCED_MANUFACTURING,
                              IndustrialStage.FRONTIER_TECH):
                    state.policy_target_sectors = STRATEGIC_SECTORS["technology_acquisition"]

            self.states[iso] = state

    def _classify_stage(self, state: CountryStructuralState,
                        endowment: pd.Series) -> IndustrialStage:
        """Classify a country's industrial stage from its capacity profile."""
        caps = state.sector_capacity
        ind = endowment["industrialization"]
        tech = endowment["tech_level"]
        resources = endowment["natural_resources"]
        gdp_pc = endowment["gdp"] / max(endowment["population"], 0.1)

        # Compute average capacity by tier
        tier_avg = {}
        for tier in range(7):
            codes = VALUE_CHAIN_TIERS.get(tier, [])
            if codes:
                tier_avg[tier] = np.mean([caps.get(c, 0) for c in codes])
            else:
                tier_avg[tier] = 0

        # Classification logic
        if tech > 0.85 and tier_avg.get(5, 0) > 40 and tier_avg.get(6, 0) > 30:
            return IndustrialStage.FRONTIER_TECH
        elif ind > 0.75 and tier_avg.get(4, 0) > 35 and tier_avg.get(5, 0) > 25:
            return IndustrialStage.ADVANCED_MANUFACTURING
        elif ind > 0.55 and tier_avg.get(3, 0) > 30:
            return IndustrialStage.HEAVY_INDUSTRY
        elif ind > 0.35 and tier_avg.get(2, 0) > 25:
            return IndustrialStage.LIGHT_MANUFACTURING
        elif resources > 0.7 and ind < 0.4:
            return IndustrialStage.RESOURCE_DEPENDENT
        else:
            return IndustrialStage.PRIMARY_EXPORT

    def run_simulation(self, projection_years: int = 10) -> pd.DataFrame:
        """
        Run the full industrialization/de-industrialization simulation.

        Phase 1: Process historical trade data (2019-2023)
        Phase 2: Project forward with dynamic structural change

        Returns a comprehensive DataFrame with structural indicators.
        """
        all_records = []

        # ---- PHASE 1: Historical Period ----
        print("=" * 70)
        print("PHASE 1: Processing Historical Trade Data (2019-2023)")
        print("=" * 70)

        for year in self.years:
            year_records = self._process_year(year, is_projection=False)
            all_records.extend(year_records)

        # ---- PHASE 2: Projection Period ----
        last_historical_year = max(self.years)
        projection_start = last_historical_year + 1
        projection_end = projection_start + projection_years

        print(f"\n{'=' * 70}")
        print(f"PHASE 2: Projecting Forward ({projection_start}-{projection_end - 1})")
        print(f"{'=' * 70}")

        for year in range(projection_start, projection_end):
            # First, generate projected trade data
            self._project_trade_data(year)

            # Then process the year
            year_records = self._process_year(year, is_projection=True)
            all_records.extend(year_records)

        return pd.DataFrame(all_records)

    def _process_year(self, year: int, is_projection: bool = False) -> List[dict]:
        """
        Process one year of trade data and update structural states.

        Core loop for each country:
        1. Calculate trade metrics from trade data
        2. Determine investment flows (from surplus/deficit)
        3. Update sector capacities
        4. Check for structural transitions
        5. Assess services trap risk
        """
        records = []

        year_trade = self.trade_data[self.trade_data["year"] == year]

        for iso in self.countries:
            state = self.states[iso]
            e = self.endowments.loc[iso]

            country_trade = year_trade[year_trade["country_iso"] == iso]

            if len(country_trade) == 0:
                continue

            # ---- Step 1: Trade Metrics ----
            metrics = self._compute_trade_metrics(country_trade, state)

            # ---- Step 2: Investment Determination ----
            investment = self._determine_investment(
                metrics, state, e, year
            )

            # ---- Step 3: Capacity Update ----
            capacity_changes = self._update_capacities(
                state, investment, metrics, e, year
            )

            # ---- Step 4: Structural Transition Check ----
            old_stage = state.industrial_stage
            old_posture = state.trade_posture

            self._check_structural_transitions(state, metrics, e, year)

            # ---- Step 5: Services Trap Risk Assessment ----
            self._assess_services_trap_risk(state, metrics, e, year)

            # ---- Step 6: Update GDP Composition ----
            self._update_gdp_composition(state, metrics, capacity_changes, year)

            # ---- Step 7: Record State ----
            transition = None
            if state.industrial_stage != old_stage:
                transition = f"{old_stage.value} → {state.industrial_stage.value}"

            posture_change = None
            if state.trade_posture != old_posture:
                posture_change = f"{old_posture.value} → {state.trade_posture.value}"

            record = {
                "year": year,
                "is_projection": is_projection,
                "country_iso": iso,
                "country_name": state.name,

                # Trade
                "total_exports": metrics["total_exports"],
                "total_imports": metrics["total_imports"],
                "trade_balance": metrics["trade_balance"],
                "mfg_exports": metrics["mfg_exports"],
                "mfg_imports": metrics["mfg_imports"],
                "mfg_trade_balance": metrics["mfg_balance"],
                "primary_exports": metrics["primary_exports"],
                "high_tech_exports": metrics["high_tech_exports"],
                "high_tech_export_share": metrics["high_tech_share"],

                # Structural
                "industrial_stage": state.industrial_stage.value,
                "trade_posture": state.trade_posture.value,
                "stage_transition": transition,
                "posture_change": posture_change,

                # Capacity
                "avg_mfg_capacity": np.mean([
                    state.sector_capacity.get(c, 0)
                    for tier in [2, 3, 4, 5, 6]
                    for c in VALUE_CHAIN_TIERS.get(tier, [])
                ]),
                "avg_hightech_capacity": np.mean([
                    state.sector_capacity.get(c, 0)
                    for tier in [5, 6]
                    for c in VALUE_CHAIN_TIERS.get(tier, [])
                ]),
                "capacity_change_total": sum(capacity_changes.values()),

                # GDP composition
                "manufacturing_share_gdp": state.manufacturing_share_gdp,
                "services_share_gdp": state.services_share_gdp,
                "primary_share_gdp": state.primary_share_gdp,

                # Cumulative
                "cumulative_trade_balance": state.cumulative_trade_balance,
                "cumulative_mfg_investment": state.cumulative_mfg_investment,
                "cumulative_capacity_erosion": state.cumulative_capacity_erosion,

                # Services trap
                "services_trap_risk": state.services_trap_risk,
                "years_in_deficit": state.years_in_deficit,
                "import_dependence_index": state.import_dependence_index,

                # Policy
                "has_industrial_policy": state.has_industrial_policy,
                "policy_effectiveness": state.policy_effectiveness,
                "policy_target_sectors": (
                    ",".join(state.policy_target_sectors)
                    if state.policy_target_sectors else ""
                ),

                # Investment
                "gross_investment": investment["gross_investment"],
                "reinvestment_from_surplus": investment["reinvestment_from_surplus"],
                "foreign_investment": investment["foreign_investment"],
                "policy_investment": investment["policy_investment"],
                "depreciation": investment["depreciation"],
                "net_investment": investment["net_investment"],
            }

            records.append(record)

            # Save to state history
            state.history[year] = record.copy()

        # Print summary for this year
        self._print_year_summary(year, records)

        return records

    def _compute_trade_metrics(self, country_trade: pd.DataFrame,
                                state: CountryStructuralState) -> dict:
        """Extract key trade metrics for structural analysis."""

        total_exports = country_trade["exports_billions_usd"].sum()
        total_imports = country_trade["imports_billions_usd"].sum()

        # Manufacturing = tiers 2-6
        mfg_codes = []
        for tier in [2, 3, 4, 5, 6]:
            mfg_codes.extend(VALUE_CHAIN_TIERS.get(tier, []))

        mfg_data = country_trade[country_trade["commodity_code"].isin(mfg_codes)]
        mfg_exports = mfg_data["exports_billions_usd"].sum()
        mfg_imports = mfg_data["imports_billions_usd"].sum()

        # High tech = tiers 5-6
        ht_codes = []
        for tier in [5, 6]:
            ht_codes.extend(VALUE_CHAIN_TIERS.get(tier, []))

        ht_data = country_trade[country_trade["commodity_code"].isin(ht_codes)]
        ht_exports = ht_data["exports_billions_usd"].sum()

        # Primary = tiers 0-1
        primary_codes = VALUE_CHAIN_TIERS.get(0, []) + VALUE_CHAIN_TIERS.get(1, [])
        primary_data = country_trade[
            country_trade["commodity_code"].isin(primary_codes)
        ]
        primary_exports = primary_data["exports_billions_usd"].sum()

        # Import dependence: how much of manufacturing demand is imported
        total_mfg_demand = mfg_imports + mfg_exports * 0.3  # Crude domestic absorption
        import_dep = mfg_imports / max(total_mfg_demand, 0.01)

        # Per-commodity net exports
        commodity_balances = {}
        for _, row in country_trade.iterrows():
            code = row["commodity_code"]
            commodity_balances[code] = (
                row["exports_billions_usd"] - row["imports_billions_usd"]
            )

        return {
            "total_exports": round(total_exports, 3),
            "total_imports": round(total_imports, 3),
            "trade_balance": round(total_exports - total_imports, 3),
            "mfg_exports": round(mfg_exports, 3),
            "mfg_imports": round(mfg_imports, 3),
            "mfg_balance": round(mfg_exports - mfg_imports, 3),
            "primary_exports": round(primary_exports, 3),
            "high_tech_exports": round(ht_exports, 3),
            "high_tech_share": round(
                ht_exports / max(total_exports, 0.01), 4
            ),
            "import_dependence": round(min(import_dep, 1.0), 4),
            "commodity_balances": commodity_balances,
        }

    def _determine_investment(self, metrics: dict,
                               state: CountryStructuralState,
                               endowment: pd.Series,
                               year: int) -> dict:
        """
        Determine investment into industrial capacity.

        LTV logic:
        - Surplus value realized through trade can be:
          (a) Reinvested in expanding production (accumulation)
          (b) Consumed by capitalists (luxury consumption)
          (c) Transferred abroad (capital flight, profit repatriation)

        - Trade surplus countries can reinvest a fraction of surplus
        - Trade deficit countries must divert resources to cover deficit
        - Industrial policy can redirect investment strategically
        """
        p = self.params

        trade_bal = metrics["trade_balance"]
        mfg_bal = metrics["mfg_balance"]
        gdp = endowment["gdp"]

        # --- Reinvestment from trade surplus ---
        if trade_bal > 0:
            # Countries with surplus can reinvest
            reinvestment_rate = p.surplus_reinvestment_rate

            # Industrial policy countries reinvest more
            if state.has_industrial_policy:
                reinvestment_rate *= (1 + 0.3 * state.policy_effectiveness)

            reinvestment = trade_bal * reinvestment_rate
        else:
            # Deficit countries: negative "reinvestment" (value drain)
            reinvestment = trade_bal * p.deficit_drain_rate

        # --- Foreign direct investment ---
        # FDI goes to countries with cheap labour + decent infrastructure
        fdi_attractiveness = (
            0.3 * endowment["labour_abundance"] *
            (1 - endowment["labour_skill"] * 0.3) +  # Cheap but not too unskilled
            0.3 * endowment["infrastructure"] +
            0.2 * (1 - state.services_trap_risk) +  # Avoid trapped economies
            0.2 * endowment["industrialization"]
        )

        if state.has_industrial_policy:
            fdi_attractiveness *= (1 + 0.2 * state.policy_effectiveness)

        # FDI as fraction of GDP
        foreign_investment = gdp * p.base_fdi_rate * fdi_attractiveness

        # --- Industrial policy investment ---
        policy_investment = 0.0
        if state.has_industrial_policy:
            # State-directed investment (% of GDP)
            policy_investment = (
                gdp * p.policy_investment_rate *
                state.policy_effectiveness
            )

        # --- Depreciation ---
        # Existing capacity depreciates over time
        avg_capacity = np.mean(list(state.sector_capacity.values()))
        depreciation = avg_capacity * p.depreciation_rate * gdp * 0.001

        # --- Gross and Net Investment ---
        gross_investment = max(reinvestment, 0) + foreign_investment + policy_investment
        net_investment = reinvestment + foreign_investment + policy_investment - depreciation

        return {
            "reinvestment_from_surplus": round(reinvestment, 3),
            "foreign_investment": round(foreign_investment, 3),
            "policy_investment": round(policy_investment, 3),
            "depreciation": round(depreciation, 3),
            "gross_investment": round(gross_investment, 3),
            "net_investment": round(net_investment, 3),
        }

    def _update_capacities(self, state: CountryStructuralState,
                           investment: dict, metrics: dict,
                           endowment: pd.Series,
                           year: int) -> dict:
        """
        Update sector capacities based on investment and trade performance.

        Key dynamics:
        - Positive net investment → capacity growth (preferentially in
          sectors where country has advantage or policy targets)
        - Negative net investment → capacity erosion (preferentially in
          sectors where country is a net importer — these get hollowed out)
        - HYSTERESIS: capacity loss is faster than capacity building
          (it takes decades to build an auto industry, years to lose it)
        """
        p = self.params
        net_inv = investment["net_investment"]
        policy_inv = investment["policy_investment"]

        capacity_changes = {}

        for code in self.commodity_codes:
            tier = COMMODITY_TO_TIER.get(code, 3)
            old_cap = state.sector_capacity.get(code, 10.0)

            commodity_balance = metrics["commodity_balances"].get(code, 0)

            # --- Base change from net investment ---
            if net_inv > 0:
                # Positive investment: allocate preferentially

                # Natural allocation: invest more where productive
                prod_score = old_cap / max(
                    np.mean(list(state.sector_capacity.values())), 1
                )

                # Policy allocation: direct investment to target sectors
                policy_boost = 0.0
                if (state.has_industrial_policy and
                    code in state.policy_target_sectors):
                    policy_boost = (
                        policy_inv * state.policy_effectiveness *
                        p.policy_targeting_precision /
                        max(len(state.policy_target_sectors), 1)
                    )

                # Investment grows capacity, but with diminishing returns
                growth = (
                    p.capacity_growth_rate *
                    (net_inv / max(endowment["gdp"] * 0.1, 1)) *
                    prod_score *
                    (1 - old_cap / 120)  # Diminishing returns near cap
                )

                # Harder to grow higher-tier sectors
                tier_difficulty = 1.0 + 0.15 * max(tier - 2, 0)
                growth /= tier_difficulty

                # Policy boost
                growth += policy_boost * 0.05

                change = growth

            else:
                # Negative investment: capacity erodes

                # Erosion is FASTER than growth (hysteresis)
                erosion_base = (
                    p.capacity_erosion_rate *
                    abs(net_inv) / max(endowment["gdp"] * 0.1, 1)
                )

                # Sectors with trade deficits erode faster
                # (foreign competition displaces domestic production)
                if commodity_balance < 0:
                    deficit_pressure = min(
                        abs(commodity_balance) / max(endowment["gdp"] * 0.01, 0.1),
                        2.0
                    )
                    erosion_base *= (1 + deficit_pressure)

                # Higher-tier sectors are more vulnerable to erosion
                # (require continuous R&D and investment to maintain)
                tier_vulnerability = 1.0 + 0.1 * max(tier - 2, 0)
                erosion_base *= tier_vulnerability

                # Industrial policy protects targeted sectors
                if (state.has_industrial_policy and
                    code in state.policy_target_sectors):
                    erosion_base *= (1 - 0.5 * state.policy_effectiveness)

                change = -erosion_base

            # Apply change with noise
            change *= (1 + 0.1 * self.rng.randn())

            new_cap = np.clip(old_cap + change, 1.0, 100.0)
            capacity_changes[code] = new_cap - old_cap
            state.sector_capacity[code] = new_cap

        # Update cumulative metrics
        state.cumulative_mfg_investment += max(investment["net_investment"], 0)
        state.cumulative_capacity_erosion += sum(
            abs(v) for v in capacity_changes.values() if v < 0
        )
        state.cumulative_trade_balance += metrics["trade_balance"]

        return capacity_changes

    def _check_structural_transitions(self, state: CountryStructuralState,
                                       metrics: dict,
                                       endowment: pd.Series,
                                       year: int):
        """
        Check if a country has transitioned between industrial stages
        or trade postures.
        """
        # --- Industrial Stage Transitions ---
        caps = state.sector_capacity

        tier_avgs = {}
        for tier in range(7):
            codes = VALUE_CHAIN_TIERS.get(tier, [])
            if codes:
                tier_avgs[tier] = np.mean([caps.get(c, 0) for c in codes])

        mfg_share = state.manufacturing_share_gdp
        ht_share = metrics["high_tech_share"]

        # Upward transitions (industrialization)
        current = state.industrial_stage

        if current == IndustrialStage.PRIMARY_EXPORT:
            if tier_avgs.get(2, 0) > 30 and mfg_share > 0.10:
                state.industrial_stage = IndustrialStage.LIGHT_MANUFACTURING

        elif current == IndustrialStage.LIGHT_MANUFACTURING:
            if tier_avgs.get(3, 0) > 35 and tier_avgs.get(4, 0) > 20 and mfg_share > 0.15:
                state.industrial_stage = IndustrialStage.HEAVY_INDUSTRY

        elif current == IndustrialStage.HEAVY_INDUSTRY:
            if tier_avgs.get(4, 0) > 40 and tier_avgs.get(5, 0) > 30 and ht_share > 0.15:
                state.industrial_stage = IndustrialStage.ADVANCED_MANUFACTURING

        elif current == IndustrialStage.ADVANCED_MANUFACTURING:
            if tier_avgs.get(5, 0) > 50 and tier_avgs.get(6, 0) > 40 and ht_share > 0.25:
                state.industrial_stage = IndustrialStage.FRONTIER_TECH

        # Downward transitions (de-industrialization)
        if current in (IndustrialStage.HEAVY_INDUSTRY,
                       IndustrialStage.ADVANCED_MANUFACTURING,
                       IndustrialStage.LIGHT_MANUFACTURING):

            # Check for premature de-industrialization
            gdp_pc = endowment["gdp"] / max(endowment["population"], 0.1)

            if (mfg_share < 0.10 and gdp_pc < 20 and
                state.years_in_deficit > 3):
                state.industrial_stage = IndustrialStage.PREMATURE_DEINDUSTRIAL

            elif (state.services_share_gdp > 0.65 and mfg_share < 0.12 and
                  state.services_trap_risk > 0.6):
                state.industrial_stage = IndustrialStage.SERVICES_DEPENDENT

        if current == IndustrialStage.RESOURCE_DEPENDENT:
            # Can escape if manufacturing develops
            if tier_avgs.get(2, 0) > 30 and tier_avgs.get(3, 0) > 20:
                state.industrial_stage = IndustrialStage.LIGHT_MANUFACTURING

        # --- Trade Posture Classification ---
        trade_bal = metrics["trade_balance"]
        mfg_bal = metrics["mfg_balance"]
        primary_exp = metrics["primary_exports"]
        total_exp = metrics["total_exports"]

        if trade_bal > 0 and mfg_bal > 0:
            state.trade_posture = TradePosture.STRATEGIC_SURPLUS
        elif trade_bal > 0 and primary_exp / max(total_exp, 0.01) > 0.6:
            state.trade_posture = TradePosture.RESOURCE_SURPLUS
        elif abs(trade_bal) < total_exp * 0.05 and mfg_share > 0.15:
            state.trade_posture = TradePosture.BALANCED_INDUSTRIAL
        elif trade_bal < 0 and mfg_share > 0.15:
            state.trade_posture = TradePosture.DEFICIT_INDUSTRIAL
        elif trade_bal < 0 and mfg_share > 0.08:
            state.trade_posture = TradePosture.DEFICIT_ERODING
        else:
            state.trade_posture = TradePosture.DEFICIT_DEPENDENT

    def _assess_services_trap_risk(self, state: CountryStructuralState,
                                    metrics: dict,
                                    endowment: pd.Series,
                                    year: int):
        """
        Assess a country's risk of falling into the services trap.

        The services trap occurs when:
        1. Persistent trade deficits in manufactured goods
        2. Manufacturing capacity erodes (can't compete)
        3. Labour shifts to non-tradeable services
        4. Services can't generate export revenue to pay for imports
        5. Country becomes dependent on external financing
        6. Any disruption to financing → crisis (the "rug pull")

        Risk factors:
        - Duration and depth of trade deficits
        - Rate of manufacturing capacity loss
        - Import dependence for essential goods
        - Lack of industrial policy
        - High services share without high-value services exports
        """
        p = self.params

        # Update deficit counter
        if metrics["trade_balance"] < 0:
            state.years_in_deficit += 1
        else:
            state.years_in_deficit = max(state.years_in_deficit - 1, 0)

        # Update import dependence
        state.import_dependence_index = metrics["import_dependence"]

        # Risk calculation (0-1)
        risk_factors = []

        # Factor 1: Persistent deficits
        deficit_persistence = min(state.years_in_deficit / 8.0, 1.0)
        risk_factors.append(("deficit_persistence", deficit_persistence, 0.20))

        # Factor 2: Manufacturing decline
        mfg_decline = max(0, 0.20 - state.manufacturing_share_gdp) / 0.15
        mfg_decline = min(mfg_decline, 1.0)
        risk_factors.append(("mfg_decline", mfg_decline, 0.20))

        # Factor 3: Import dependence
        risk_factors.append(("import_dependence", metrics["import_dependence"], 0.15))

        # Factor 4: Capacity erosion rate
        erosion_rate = min(
            state.cumulative_capacity_erosion / max(endowment["gdp"] * 0.5, 1),
            1.0
        )
        risk_factors.append(("erosion_rate", erosion_rate, 0.15))

        # Factor 5: Services bloat without high-value exports
        services_bloat = max(0, state.services_share_gdp - 0.55) / 0.30
        services_bloat = min(services_bloat, 1.0)
        # Mitigated by high-tech services (not modeled here, use proxy)
        gdp_pc = endowment["gdp"] / max(endowment["population"], 0.1)
        if gdp_pc > 40:  # Rich service economies (US, UK) are less at risk
            services_bloat *= 0.5
        risk_factors.append(("services_bloat", services_bloat, 0.15))

        # Factor 6: Lack of industrial policy
        no_policy = 1.0 - (
            state.policy_effectiveness if state.has_industrial_policy else 0
        )
        risk_factors.append(("no_industrial_policy", no_policy, 0.10))

        # Factor 7: Low cumulative investment
        low_investment = 1.0 - min(
            state.cumulative_mfg_investment / max(endowment["gdp"] * 0.3, 1),
            1.0
        )
        risk_factors.append(("low_investment", low_investment, 0.05))

        # Weighted sum
        risk = sum(score * weight for _, score, weight in risk_factors)
        risk = np.clip(risk, 0, 1)

        # Momentum: risk tends to persist and compound
        old_risk = state.services_trap_risk
        risk = 0.7 * risk + 0.3 * old_risk  # Smoothing

        state.services_trap_risk = round(risk, 4)

    def _update_gdp_composition(self, state: CountryStructuralState,
                                 metrics: dict,
                                 capacity_changes: dict,
                                 year: int):
        """
        Update the GDP composition (manufacturing/services/primary shares)
        based on capacity changes and trade performance.
        """
        # Manufacturing share responds to capacity changes
        mfg_codes = []
        for tier in [2, 3, 4, 5, 6]:
            mfg_codes.extend(VALUE_CHAIN_TIERS.get(tier, []))

        avg_mfg_cap_change = np.mean([
            capacity_changes.get(c, 0) for c in mfg_codes
        ])

        # Manufacturing share adjusts slowly
        mfg_adjustment = avg_mfg_cap_change * 0.002

        # Trade balance effect on manufacturing
        if metrics["mfg_balance"] < 0:
            # Import competition reduces manufacturing share
            mfg_pressure = min(
                abs(metrics["mfg_balance"]) /
                max(metrics["total_imports"] * 0.3, 1),
                0.02
            )
            mfg_adjustment -= mfg_pressure * 0.01

        state.manufacturing_share_gdp = np.clip(
            state.manufacturing_share_gdp + mfg_adjustment,
            0.03, 0.38
        )

        # Primary share: slowly declining for most countries
        primary_decline = 0.002  # Secular trend
        if state.has_industrial_policy:
            primary_decline *= 1.5  # Policy accelerates structural change

        state.primary_share_gdp = np.clip(
            state.primary_share_gdp - primary_decline,
            0.02, 0.55
        )

        # Services is the residual
        state.services_share_gdp = np.clip(
            1.0 - state.manufacturing_share_gdp - state.primary_share_gdp,
            0.20, 0.85
        )

        # Normalize to sum to 1
        total = (state.manufacturing_share_gdp + state.services_share_gdp +
                 state.primary_share_gdp)
        state.manufacturing_share_gdp /= total
        state.services_share_gdp /= total
        state.primary_share_gdp /= total

    def _project_trade_data(self, year: int):
        """
        Generate projected trade data for future years based on
        current structural states and capacity trajectories.
        """
        last_year = year - 1
        last_data = self.trade_data[self.trade_data["year"] == last_year]

        new_rows = []

        for iso in self.countries:
            state = self.states[iso]
            e = self.endowments.loc[iso]

            country_last = last_data[last_data["country_iso"] == iso]

            for _, row in country_last.iterrows():
                code = row["commodity_code"]
                tier = COMMODITY_TO_TIER.get(code, 3)

                # Capacity-based growth
                old_cap = state.sector_capacity.get(code, 10)

                # Export growth follows capacity trajectory
                cap_ratio = old_cap / max(
                    np.mean(list(state.sector_capacity.values())), 1
                )

                export_growth = 1.0 + 0.03 * (cap_ratio - 1)
                import_growth = 1.0 + 0.02

                # Structural effects
                if state.trade_posture in (TradePosture.DEFICIT_ERODING,
                                           TradePosture.DEFICIT_DEPENDENT):
                    # Eroding countries: exports fall, imports persist
                    if tier >= 2:  # Manufacturing
                        export_growth *= 0.97
                        import_growth *= 1.02

                if state.trade_posture == TradePosture.STRATEGIC_SURPLUS:
                    if tier >= 3:  # Industrial exports grow faster
                        export_growth *= 1.04

                # Industrial policy boost
                if (state.has_industrial_policy and
                    code in state.policy_target_sectors):
                    export_growth *= (1 + 0.02 * state.policy_effectiveness)
                    import_growth *= (1 - 0.01 * state.policy_effectiveness)

                # Noise
                export_growth *= (1 + 0.03 * self.rng.randn())
                import_growth *= (1 + 0.03 * self.rng.randn())

                new_row = row.copy()
                new_row["year"] = year
                new_row["exports_billions_usd"] = round(
                    row["exports_billions_usd"] * max(export_growth, 0.8), 3
                )
                new_row["imports_billions_usd"] = round(
                    row["imports_billions_usd"] * max(import_growth, 0.8), 3
                )
                new_row["net_exports_billions_usd"] = round(
                    new_row["exports_billions_usd"] -
                    new_row["imports_billions_usd"], 3
                )

                new_rows.append(new_row)

        new_df = pd.DataFrame(new_rows)
        self.trade_data = pd.concat([self.trade_data, new_df], ignore_index=True)

    def _print_year_summary(self, year: int, records: List[dict]):
        """Print a compact summary for the year."""
        print(f"\n--- {year} ---")

        # Count transitions
        stage_transitions = [r for r in records if r["stage_transition"]]
        posture_changes = [r for r in records if r["posture_change"]]

        if stage_transitions:
            print(f"  Industrial stage transitions:")
            for t in stage_transitions[:5]:
                print(f"    {t['country_name']}: {t['stage_transition']}")
            if len(stage_transitions) > 5:
                print(f"    ... and {len(stage_transitions) - 5} more")

        if posture_changes:
            print(f"  Trade posture changes:")
            for t in posture_changes[:5]:
                print(f"    {t['country_name']}: {t['posture_change']}")
            if len(posture_changes) > 5:
                print(f"    ... and {len(posture_changes) - 5} more")

        # Top risks
        high_risk = sorted(
            [r for r in records if r["services_trap_risk"] > 0.5],
            key=lambda x: -x["services_trap_risk"]
        )[:5]

        if high_risk:
            print(f"  Highest services trap risk:")
            for r in high_risk:
                print(f"    {r['country_name']}: {r['services_trap_risk']:.2f} "
                      f"(deficit years: {r['years_in_deficit']})")


# ============================================================
# 5. SIMULATION PARAMETERS
# ============================================================

@dataclass
class IndustrializationParams:
    """Tunable parameters for the simulation."""

    # Investment rates
    surplus_reinvestment_rate: float = 0.25   # % of trade surplus reinvested
    deficit_drain_rate: float = 0.15          # % of deficit that drains capacity
    base_fdi_rate: float = 0.008              # FDI as % of GDP (base)
    policy_investment_rate: float = 0.02      # State industrial investment as % of GDP

    # Capacity dynamics
    capacity_growth_rate: float = 2.0         # Base capacity growth per unit investment
    capacity_erosion_rate: float = 3.5        # Erosion rate (FASTER than growth = hysteresis)
    depreciation_rate: float = 0.03           # Annual depreciation of capacity

    # Policy
    policy_targeting_precision: float = 0.6   # How well policy targets right sectors

    # Structural thresholds
    mfg_share_deindustrial_threshold: float = 0.10  # Below this = de-industrializing
    services_trap_entry_threshold: float = 0.65      # Risk above this = entering trap


# ============================================================
# 6. VISUALIZATION & REPORTING
# ============================================================

def generate_structural_report(results: pd.DataFrame,
                                focus_countries: Optional[List[str]] = None
                                ) -> str:
    """Generate a human-readable report of structural dynamics."""

    if focus_countries is None:
        focus_countries = [
            "USA", "CHN", "DEU", "JPN", "GBR", "IND", "BRA", "KOR",
            "MEX", "VNM", "NGA", "BGD", "TUR", "IDN", "ZAF", "ARG",
            "ETH", "THA", "EGY", "RUS"
        ]

    report = []
    report.append("=" * 80)
    report.append("STRUCTURAL DYNAMICS REPORT: INDUSTRIALIZATION & DE-INDUSTRIALIZATION")
    report.append("=" * 80)

    years = sorted(results["year"].unique())
    first_year = min(years)
    last_year = max(years)

    for iso in focus_countries:
        country_data = results[results["country_iso"] == iso]
        if len(country_data) == 0:
            continue

        first = country_data[country_data["year"] == first_year].iloc[0]
        last = country_data[country_data["year"] == last_year].iloc[0]

        report.append(f"\n{'─' * 60}")
        report.append(f"  {last['country_name']} ({iso})")
        report.append(f"{'─' * 60}")

        report.append(f"  Industrial Stage: {first['industrial_stage']} → "
                      f"{last['industrial_stage']}")
        report.append(f"  Trade Posture:    {first['trade_posture']} → "
                      f"{last['trade_posture']}")

        report.append(f"\n  Manufacturing Share of GDP: "
                      f"{first['manufacturing_share_gdp']:.1%} → "
                      f"{last['manufacturing_share_gdp']:.1%}")
        report.append(f"  Services Share of GDP:      "
                      f"{first['services_share_gdp']:.1%} → "
                      f"{last['services_share_gdp']:.1%}")

        report.append(f"\n  Trade Balance ({first_year}): "
                      f"${first['trade_balance']:+.1f}B")
        report.append(f"  Trade Balance ({last_year}): "
                      f"${last['trade_balance']:+.1f}B")
        report.append(f"  Cumulative Balance: "
                      f"${last['cumulative_trade_balance']:+.1f}B")

        report.append(f"\n  High-Tech Export Share: "
                      f"{first['high_tech_export_share']:.1%} → "
                      f"{last['high_tech_export_share']:.1%}")
        report.append(f"  Avg Mfg Capacity: "
                      f"{first['avg_mfg_capacity']:.1f} → "
                      f"{last['avg_mfg_capacity']:.1f}")

        report.append(f"\n  Services Trap Risk: "
                      f"{first['services_trap_risk']:.2f} → "
                      f"{last['services_trap_risk']:.2f}")

        if last["services_trap_risk"] > 0.5:
            report.append(f"  ⚠️  HIGH RISK of services trap entry")
        elif last["services_trap_risk"] > 0.3:
            report.append(f"  ⚡ ELEVATED risk of services trap")

        # Check for transitions
        transitions = country_data[
            country_data["stage_transition"].notna() &
            (country_data["stage_transition"] != "")
        ]
        if len(transitions) > 0:
            report.append(f"\n  Stage Transitions:")
            for _, t in transitions.iterrows():
                report.append(f"    {t['year']}: {t['stage_transition']}")

        if last["has_industrial_policy"]:
            report.append(f"\n  Industrial Policy: Active "
                         f"(effectiveness: {last['policy_effectiveness']:.2f})")
            report.append(f"  Target Sectors: {last['policy_target_sectors']}")

    report.append(f"\n{'=' * 80}")
    report.append("SUMMARY STATISTICS")
    report.append(f"{'=' * 80}")

    last_year_data = results[results["year"] == last_year]

    # Stage distribution
    report.append(f"\nIndustrial Stage Distribution ({last_year}):")
    stage_counts = last_year_data["industrial_stage"].value_counts()
    for stage, count in stage_counts.items():
        report.append(f"  {stage}: {count} countries")

    # Posture distribution
    report.append(f"\nTrade Posture Distribution ({last_year}):")
    posture_counts = last_year_data["trade_posture"].value_counts()
    for posture, count in posture_counts.items():
        report.append(f"  {posture}: {count} countries")

    # Services trap risk
    report.append(f"\nServices Trap Risk Distribution ({last_year}):")
    high = len(last_year_data[last_year_data["services_trap_risk"] > 0.5])
    medium = len(last_year_data[
        (last_year_data["services_trap_risk"] > 0.3) &
        (last_year_data["services_trap_risk"] <= 0.5)
    ])
    low = len(last_year_data[last_year_data["services_trap_risk"] <= 0.3])
    report.append(f"  High risk (>0.5):   {high} countries")
    report.append(f"  Medium risk (0.3-0.5): {medium} countries")
    report.append(f"  Low risk (<0.3):    {low} countries")

    return "\n".join(report)


def generate_divergence_analysis(results: pd.DataFrame) -> pd.DataFrame:
    """
    Analyze the divergence between industrializing and
    de-industrializing countries over time.

    Groups countries into trajectories and computes aggregate metrics.
    """
    years = sorted(results["year"].unique())
    last_year = max(years)
    first_year = min(years)

    # Classify trajectory for each country
    trajectories = []

    for iso in results["country_iso"].unique():
        cd = results[results["country_iso"] == iso].sort_values("year")
        if len(cd) < 2:
            continue

        first = cd.iloc[0]
        last = cd.iloc[-1]

        mfg_change = last["manufacturing_share_gdp"] - first["manufacturing_share_gdp"]
        cap_change = last["avg_mfg_capacity"] - first["avg_mfg_capacity"]
        ht_change = last["high_tech_export_share"] - first["high_tech_export_share"]

        if mfg_change > 0.02 and cap_change > 2:
            trajectory = "industrializing"
        elif mfg_change > 0 and cap_change > 0:
            trajectory = "stable_industrial"
        elif mfg_change < -0.02 and last["services_trap_risk"] > 0.4:
            trajectory = "services_trap_path"
        elif mfg_change < -0.01:
            trajectory = "de-industrializing"
        else:
            trajectory = "stable"

        trajectories.append({
            "country_iso": iso,
            "country_name": last["country_name"],
            "trajectory": trajectory,
            "mfg_share_change": round(mfg_change, 4),
            "capacity_change": round(cap_change, 2),
            "ht_share_change": round(ht_change, 4),
            "final_mfg_share": round(last["manufacturing_share_gdp"], 4),
            "final_services_trap_risk": round(last["services_trap_risk"], 4),
            "final_trade_balance": round(last["trade_balance"], 2),
            "cumulative_balance": round(last["cumulative_trade_balance"], 2),
        })

    return pd.DataFrame(trajectories)


# ============================================================
# 7. MAIN EXECUTION
# ============================================================

if __name__ == "__main__":

    # --- Step 1: Generate base trade data ---
    print("Generating base LTV trade data...")
    base_model = LTVTradeModel(COUNTRIES, seed=42)
    trade_data = base_model.generate_trade_data()

    print(f"Base trade data: {len(trade_data):,} records")

    # --- Step 2: Run industrialization simulation ---
    print("\nInitializing Industrialization Engine...")
    engine = IndustrializationEngine(
        trade_data=trade_data,
        endowments=base_model.endowments,
        seed=42
    )

    # Run with 10 years of projection (2024-2033)
    print("\nRunning simulation...")
    results = engine.run_simulation(projection_years=10)

    print(f"\nSimulation complete: {len(results):,} country-year records")

    # --- Step 3: Generate reports ---
    report = generate_structural_report(results)
    print(report)

    # --- Step 4: Divergence analysis ---
    divergence = generate_divergence_analysis(results)

    print("\n" + "=" * 80)
    print("TRAJECTORY CLASSIFICATION")
    print("=" * 80)

    for trajectory in ["industrializing", "stable_industrial", "stable",
                       "de-industrializing", "services_trap_path"]:
        group = divergence[divergence["trajectory"] == trajectory]
        print(f"\n{trajectory.upper()} ({len(group)} countries):")

        if len(group) > 0:
            # Show top 8 by absolute mfg_share_change
            top = group.nlargest(8, "mfg_share_change" if trajectory == "industrializing"
                                else "final_services_trap_risk")
            for _, row in top.iterrows():
                print(f"  {row['country_name']:30s} "
                      f"Mfg: {row['mfg_share_change']:+.2%}  "
                      f"Cap: {row['capacity_change']:+5.1f}  "
                      f"Trap Risk: {row['final_services_trap_risk']:.2f}  "
                      f"Cum.Bal: ${row['cumulative_balance']:+8.1f}B")

    # --- Step 5: Save results ---
    results.to_csv("industrialization_dynamics.csv", index=False)
    divergence.to_csv("divergence_analysis.csv", index=False)

    with open("structural_report.txt", "w") as f:
        f.write(report)

    print("\n" + "=" * 80)
    print("Files saved:")
    print("  - industrialization_dynamics.csv (full time series)")
    print("  - divergence_analysis.csv (trajectory classification)")
    print("  - structural_report.txt (narrative report)")
    print("=" * 80)

    # --- Step 6: Quick diagnostic plots data ---
    print("\n" + "=" * 80)
    print("DIAGNOSTIC: Manufacturing Share Trajectories (Selected Countries)")
    print("=" * 80)

    diagnostic_countries = ["CHN", "USA", "DEU", "IND", "BRA", "GBR",
                           "KOR", "VNM", "NGA", "MEX", "ARG", "BGD"]

    for iso in diagnostic_countries:
        cd = results[results["country_iso"] == iso][
            ["year", "manufacturing_share_gdp", "services_trap_risk",
             "trade_balance", "industrial_stage"]
        ]
        if len(cd) == 0:
            continue

        name = results[results["country_iso"] == iso]["country_name"].iloc[0]
        print(f"\n  {name}:")
        print(f"  {'Year':>6s}  {'Mfg%':>6s}  {'TrapRisk':>8s}  "
              f"{'TradeBal':>10s}  {'Stage'}")
        print(f"  {'─' * 55}")

        for _, row in cd.iterrows():
            print(f"  {row['year']:>6.0f}  {row['manufacturing_share_gdp']:>6.1%}  "
                  f"{row['services_trap_risk']:>8.3f}  "
                  f"${row['trade_balance']:>+9.1f}B  "
                  f"{row['industrial_stage']}")

Generating base LTV trade data...
Generating trade data for 2019...
Generating trade data for 2020...
Generating trade data for 2021...
Generating trade data for 2022...
Generating trade data for 2023...
Base trade data: 17,595 records

Initializing Industrialization Engine...

Running simulation...
PHASE 1: Processing Historical Trade Data (2019-2023)

--- 2019 ---
  Industrial stage transitions:
    China: advanced_mfg → frontier_tech
    India: light_manufacturing → heavy_industry
    France: advanced_mfg → frontier_tech
    Italy: advanced_mfg → frontier_tech
    Brazil: heavy_industry → advanced_mfg
    ... and 78 more
  Trade posture changes:
    United States: balanced_industrial → deficit_industrial
    China: balanced_industrial → strategic_surplus
    Japan: balanced_industrial → deficit_industrial
    Germany: balanced_industrial → deficit_industrial
    United Kingdom: balanced_industrial → deficit_industrial
    ... and 124 more

--- 2020 ---
  Industrial stage transitions